# Emission lines

In addition to creating and manipulating spectral energy distributions, `synthesizer` can also create `Line` objects, or more usefully collections of emission lines, `LineCollection` objects, that can be further analysed or manipulated. 

Like spectral energy distributions lines can be extracted directly from `Grid` objects or generated by `Galaxy` objects.

## Extracting lines from `Grid` objects

Grids that have been post-processed through CLOUDY also contain information on nebular emission lines. These can be loaded like regular grids, but there are a number of additional methods for working with lines as demonstrated in these examples.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import synthesizer.line_ratios as line_ratios
from synthesizer.grid import Grid
from synthesizer.line import (
    get_diagram_labels,
)

Let's first introduce the `line_ratios` module. This contains a set of useful definitions.

In [ ]:
# the ID of H-alpha
print(line_ratios.Ha)

# the available in-built line ratios ...
print(line_ratios.available_ratios)

# ... and diagrams.
print(line_ratios.available_diagrams)

Next let's initialise a grid:

In [ ]:
grid_dir = "../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

We can easily get a list of the available lines:

In [ ]:
print(grid.available_lines)

This is also reported if we give the grid to the `print` function:

In [ ]:
print(grid)

Let's choose an age and metallicity we want to get predictions. The in-built method will find the nearest grid point:

In [ ]:
log10age = 6.0  # log10(age/yr)
metallicity = 0.01
# find nearest grid point
grid_point = grid.get_grid_point((log10age, metallicity))

Let's get information on a single line, in this case H-beta:

In [ ]:
line_id = line_ratios.Hb
line = grid.get_line(grid_point, line_id)
print(line)

We can do this for a combination of lines (e.g. a doublet) like this. Note: this sums the contribution of each line. If you want separate lines using the `get_lines` method described below.

In [ ]:
line = grid.get_line(
    grid_point, [line_ratios.Hb, line_ratios.O3r, line_ratios.O3b]
)
print(line)

We can also create a `LineCollection` a collection of lines which have methods for calculating ratios and diagrams. By default this will create a collection for all available lines but you can also specify which lines you want.

In [ ]:
lines = grid.get_lines(grid_point)
print(lines)

We can measure some predifined line ratios:

In [ ]:
ratio_id = "BalmerDecrement"
ratio = lines.get_ratio(ratio_id)
print(f"{ratio_id}: {ratio:.2f}")

Or loop over all pre-defined ratios:

In [ ]:
for ratio_id in lines.available_ratios:
    ratio = lines.get_ratio(ratio_id)
    print(f"{ratio_id}: {ratio:.2f}")

We can also easily measure the ratio of an arbitrary set of lines:

In [ ]:
lines.get_ratio(["Ne 4 1601.45A", "He 2 1640.41A"])

In [ ]:
lines.get_ratio(["Ne 4 1601.45A, He 2 1640.41A", "O 3 1660.81A"])

We can plot a ratio against metallicity by looping over the metallicity grid:

In [ ]:
ratio_id = "R23"
ia = 0  # 1 Myr old for test grid
ratios = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    ratios.append(lines.get_ratio(ratio_id))

Zsun = grid.metallicity / 0.0124
plt.plot(Zsun, ratios)
plt.xlim([0.01, 1])
plt.ylim([1, 20])
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$Z/Z_{\odot}$")
plt.ylabel(rf"{ratio_id}")
# plt.ylabel(rf'${get_ratio_label(ratio_id)}$')
plt.show()

We can also generate "diagrams" pairs of line ratios like the BPT diagram.

The `line_ratios` also contains some hardcoded literature dividing lines (e.g. Kewley / Kauffmann) that we can use.

In [ ]:
diagram_id = "BPT-NII"
ia = 0  # 1 Myr old for test grid
x = []
y = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    x_, y_ = lines.get_diagram(diagram_id)
    x.append(x_)
    y.append(y_)


# plot the Kewley SF/AGN dividing line

logNII_Ha = np.arange(-2.0, 1.0, 0.01)
logOIII_Hb = line_ratios.get_bpt_kewley01(logNII_Ha)
plt.plot(10**logNII_Ha, 10**logOIII_Hb, c="k", lw="2", alpha=0.3)

plt.plot(x, y)
plt.xlim([0.01, 10])
plt.ylim([0.05, 20])
plt.xscale("log")
plt.yscale("log")

# grab x and y labels, this time use "fancy" label ids
xlabel, ylabel = get_diagram_labels(diagram_id)

plt.xlabel(rf"${xlabel}$")
plt.ylabel(rf"${ylabel}$")
plt.show()

## Lines from `Galaxy` objects

Of course, you're mainly going to want to generate lines from components of a `Galaxy` (i.e. parametric or particle based stars or black holes). To do this you can utlise a component's `get_line_intrinsic` (intrinsic line emission), `get_line_screen` (line emission with a simple dust screen) or `get_line_attenuated` (line emission with more complex dust emission split into a nebular and ISM component) methods. These methods are analogous to those on a grid with the extra component specific processes, i.e. they return a `LineCollection` containing the requested lines which can either be singular, doublets, triplets or more.

In [ ]:
from synthesizer.parametric import SFH, Stars, ZDist
from unyt import Myr

# Make a parametric galaxy
stellar_mass = 10**12
sfh = SFH.Constant(duration=100 * Myr)
metal_dist = ZDist.Normal(mean=0.01, sigma=0.05)
stars = Stars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=stellar_mass,
)

lc_intrinsic = stars.get_line_intrinsic(grid, line_ids="O 3 4363.21A")
print(lc_intrinsic)
lc_screen = stars.get_line_screen(
    grid, line_ids=("H 1 4340.46A, O 3 4958.91A", "O 3 5006.84A"), tau_v=0.5
)
print(lc_screen)
lc_att = stars.get_line_attenuated(
    grid,
    line_ids=["Ne 4 1601.45A", "He 2 1640.41A", "O 3 5006.84A"],
    tau_v_nebular=0.7,
    tau_v_stellar=0.5,
)
print(lc_att)

In the case of a particle based galaxy you can either get the integrated line emission...

In [ ]:
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG

# Get the stars from a particle based galaxy
stars = load_CAMELS_IllustrisTNG(
    "../../tests/data/",
    snap_name="camels_snap.hdf5",
    fof_name="camels_subhalo.hdf5",
    physical=True,
)[0].stars

lc_intrinsic = stars.get_line_intrinsic(grid, line_ids="O 3 4363.21A")
print(lc_intrinsic)
lc_screen = stars.get_line_screen(
    grid, line_ids=("H 1 4340.46A, O 3 4958.91A", "O 3 5006.84A"), tau_v=0.5
)
print(lc_screen)
lc_att = stars.get_line_attenuated(
    grid,
    line_ids=["Ne 4 1601.45A", "He 2 1640.41A", "O 3 5006.84A"],
    tau_v_nebular=0.7,
    tau_v_stellar=0.5,
)
print(lc_att)

Or per particle line emission.

In [ ]:
lc_intrinsic = stars.get_particle_line_intrinsic(grid, line_ids="O 3 4363.21A")
print(lc_intrinsic)
lc_screen = stars.get_particle_line_screen(
    grid, line_ids=("H 1 4340.46A, O 3 4958.91A", "O 3 5006.84A"), tau_v=0.5
)
print(lc_screen)
lc_att = stars.get_particle_line_attenuated(
    grid,
    line_ids=["Ne 4 1601.45A", "He 2 1640.41A", "O 3 5006.84A"],
    tau_v_nebular=0.7,
    tau_v_stellar=0.5,
)
print(lc_att)